### 표정 분류

https://drive.google.com/file/d/1lpwQNwijBfaSr8knSNHKWu5KUmzYWU9d/view?usp=drive_link

In [1]:
from glob import glob
import os 

root = './datasets/face/original'

directories =  glob(os.path.join(root, '*'))
directory_names = []

for directory in directories:
    directory_names.append(directory[directory.rindex('\\') +1:])

print(directory_names)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [2]:
root = './datasets/face/original/'

for name in directory_names:
    for i, file_name in enumerate(os.listdir(os.path.join(root, name))):
        old_file = os.path.join(root+ name +'/',file_name)
        new_file = os.path.join(root+name+'/',name+str(i+1)+'.png')

        os.rename(old_file, new_file)

In [3]:
from glob import glob
import os 

root = './datasets/face/test'

directories =  glob(os.path.join(root, '*'))
directory_names = []

for directory in directories:
    directory_names.append(directory[directory.rindex('\\') +1:])

print(directory_names)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [4]:
root = './datasets/face/test/'

for name in directory_names:
    for i, file_name in enumerate(os.listdir(os.path.join(root, name))):
        old_file = os.path.join(root+ name + '/' ,file_name)
        new_file = os.path.join(root+ name + '/' , name + str(i+1) +'.png')

        os.rename(old_file, new_file)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_data_generator = ImageDataGenerator(rescale=1./255)
generator = image_data_generator.flow_from_directory(root, target_size=(150,150), batch_size=32, class_mode='categorical')
print(generator.class_indices)

Found 7178 images belonging to 7 classes.
{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [6]:
import pandas as pd
test_df = pd.DataFrame({'file_paths': generator.filepaths, 'targets':generator.classes})
test_df.file_paths

0              ./datasets/face/test/angry\angry1.png
1             ./datasets/face/test/angry\angry10.png
2            ./datasets/face/test/angry\angry100.png
3            ./datasets/face/test/angry\angry101.png
4            ./datasets/face/test/angry\angry102.png
                            ...                     
7173    ./datasets/face/test/surprise\surprise95.png
7174    ./datasets/face/test/surprise\surprise96.png
7175    ./datasets/face/test/surprise\surprise97.png
7176    ./datasets/face/test/surprise\surprise98.png
7177    ./datasets/face/test/surprise\surprise99.png
Name: file_paths, Length: 7178, dtype: object

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

root = './datasets/face/original/'

image_data_generator = ImageDataGenerator(rescale=1./255)
generator = image_data_generator.flow_from_directory(root, target_size=(150,150), batch_size=32, class_mode='categorical')
print(generator.class_indices)

Found 28709 images belonging to 7 classes.
{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [8]:
import pandas as pd
train_df = pd.DataFrame({'file_paths': generator.filepaths, 'targets':generator.classes})
train_df

,file_paths,targets
0,./datasets/face/original/angry\angry1.png,0
1,./datasets/face/original/angry\angry10.png,0
2,./datasets/face/original/angry\angry100.png,0
3,./datasets/face/original/angry\angry1000.png,0
4,./datasets/face/original/angry\angry1001.png,0
...,...,...
28704,./datasets/face/original/surprise\surprise995.png,6
28705,./datasets/face/original/surprise\surprise996.png,6
28706,./datasets/face/original/surprise\surprise997.png,6
28707,./datasets/face/original/surprise\surprise998.png,6


In [9]:
train_df.loc[:, 'file_paths'] = train_df.file_paths.apply(lambda x: x.replace('\\','/')).reset_index(drop=True)
test_df.loc[:, 'file_paths'] = test_df.file_paths.apply(lambda x: x.replace('\\','/')).reset_index(drop=True)
display(train_df.file_paths)
display(test_df.file_paths)

0                ./datasets/face/original/angry/angry1.png
1               ./datasets/face/original/angry/angry10.png
2              ./datasets/face/original/angry/angry100.png
3             ./datasets/face/original/angry/angry1000.png
4             ./datasets/face/original/angry/angry1001.png
                               ...                        
28704    ./datasets/face/original/surprise/surprise995.png
28705    ./datasets/face/original/surprise/surprise996.png
28706    ./datasets/face/original/surprise/surprise997.png
28707    ./datasets/face/original/surprise/surprise998.png
28708    ./datasets/face/original/surprise/surprise999.png
Name: file_paths, Length: 28709, dtype: object

0              ./datasets/face/test/angry/angry1.png
1             ./datasets/face/test/angry/angry10.png
2            ./datasets/face/test/angry/angry100.png
3            ./datasets/face/test/angry/angry101.png
4            ./datasets/face/test/angry/angry102.png
                            ...                     
7173    ./datasets/face/test/surprise/surprise95.png
7174    ./datasets/face/test/surprise/surprise96.png
7175    ./datasets/face/test/surprise/surprise97.png
7176    ./datasets/face/test/surprise/surprise98.png
7177    ./datasets/face/test/surprise/surprise99.png
Name: file_paths, Length: 7178, dtype: object

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val =\
train_test_split(train_df.file_paths, train_df.targets, stratify=train_df.targets, test_size=0.2, random_state=124)

print(y_train.value_counts())
print(y_val.value_counts())

targets
3    5772
4    3972
5    3864
2    3277
0    3196
6    2537
1     349
Name: count, dtype: int64
targets
3    1443
4     993
5     966
2     820
0     799
6     634
1      87
Name: count, dtype: int64


In [11]:
import shutil

root = './datasets/face/'

for file_path in X_train:
    print(file_path)
    face_dir = file_path[len(root+'original/'):file_path.rindex('/')]
    print(face_dir)
    destination = os.path.join(root,'train/'+face_dir)
    if not os.path.exists(destination):
        os.makedirs(destination)
    shutil.copy2(file_path, destination)

./datasets/face/original/sad/sad4254.png
sad
./datasets/face/original/sad/sad4586.png
sad
./datasets/face/original/neutral/neutral1861.png
neutral
./datasets/face/original/angry/angry3888.png
angry
./datasets/face/original/sad/sad181.png
sad
./datasets/face/original/surprise/surprise420.png
surprise
./datasets/face/original/happy/happy213.png
happy
./datasets/face/original/fear/fear136.png
fear
./datasets/face/original/fear/fear2010.png
fear
./datasets/face/original/neutral/neutral2628.png
neutral
./datasets/face/original/fear/fear2445.png
fear
./datasets/face/original/fear/fear2888.png
fear
./datasets/face/original/angry/angry3927.png
angry
./datasets/face/original/surprise/surprise483.png
surprise
./datasets/face/original/angry/angry2781.png
angry
./datasets/face/original/angry/angry3083.png
angry
./datasets/face/original/sad/sad3033.png
sad
./datasets/face/original/neutral/neutral1687.png
neutral
./datasets/face/original/fear/fear3961.png
fear
./datasets/face/original/surprise/surpr

In [12]:
import shutil

root = './datasets/face/'

for file_path in X_val:
    print(file_path)
    face_dir = file_path[len(root+'original/'):file_path.rindex('/')]
    print(face_dir)
    destination = os.path.join(root,'validation/'+face_dir)
    if not os.path.exists(destination):
        os.makedirs(destination)
    shutil.copy2(file_path, destination)

./datasets/face/original/happy/happy2734.png
happy
./datasets/face/original/sad/sad891.png
sad
./datasets/face/original/neutral/neutral2226.png
neutral
./datasets/face/original/happy/happy3212.png
happy
./datasets/face/original/neutral/neutral547.png
neutral
./datasets/face/original/sad/sad1075.png
sad
./datasets/face/original/happy/happy5208.png
happy
./datasets/face/original/fear/fear2691.png
fear
./datasets/face/original/happy/happy4886.png
happy
./datasets/face/original/neutral/neutral629.png
neutral
./datasets/face/original/angry/angry3668.png
angry
./datasets/face/original/sad/sad1799.png
sad
./datasets/face/original/sad/sad176.png
sad
./datasets/face/original/surprise/surprise102.png
surprise
./datasets/face/original/surprise/surprise454.png
surprise
./datasets/face/original/sad/sad2786.png
sad
./datasets/face/original/disgust/disgust328.png
disgust
./datasets/face/original/happy/happy1228.png
happy
./datasets/face/original/neutral/neutral3850.png
neutral
./datasets/face/origina